# Where to open a Coffee Shop in London

## Business Issue: Market sizing

A young enterpreneur desires to open an Coffee Shop in the whereabouts of London. However, in order to fill its business plan it is required to make an estimation of the expected demand (taking competition into account), so that more precise decision about the business set up can be taken.

## Data Employed: Foursquare and Wikipedia

To answer its question, we will be using a combination of data from Foursquare and Wikipedia. The former will provide the information we need regarding competition, while wikipedia will provide us a size of the Bourough in terms of population, which will be assumed to be the expected demand. Thus combining the two data we can calculate the ratio borough population over competition, which will return the expected demand of the business. This, of course, is a resoning based upon several hypothesis, such as equal market shares across competitors, infinite production capacity of all the same,and  willingness (and possibility) to purchase of the entire population.

## Methodology


To begin the creation of our table we need to import all the libraries (the ones that are going to be required in the upcoming analysis are included)

In [1]:
#Libraries
import pandas as pd
import numpy as np

import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import re
import numpy as np

! pip install folium==0.5.0
import folium

We then proceed to scrape the data we need from wikipedia, in order to obtain a table with all the features we need. Then, we proceed to polish the data so that it will fit our scope.

In [2]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs')[0]
df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [3]:
pd.Series(df['Borough']=='Not assigned').value_counts()

False    32
Name: Borough, dtype: int64

In [4]:
df=df.drop(columns=['Inner', 'Status', 'Local authority', 'Political control','Headquarters','Nr. in map','Co-ordinates'])
df['Borough']=[re.sub(r'\[.*\]', '', borough) for borough in df['Borough']]
df=df.rename(columns={"Population (2019 est)[1]": "Population"})
df

,Borough,Area (sq mi),Population
0,Barking and Dagenham,13.93,212906
1,Barnet,33.49,395896
2,Bexley,23.38,248287
3,Brent,16.70,329771
4,Bromley,57.97,332336
5,Camden,8.40,270029
6,Croydon,33.41,386710
7,Ealing,21.44,341806
8,Enfield,31.74,333794
9,Greenwich,18.28,287942


In the polishing, we trasnform the feature Area (sq mi) into a km measure and we compute its root. This feature will eventually be used as the radius to scan the area surrounding the center of the Borough (little generality should be lost given the shape of the Boroughs).

In [9]:
df['Radius']=round(np.sqrt(df['Area (sq mi)']*1609.344),0)
df

,Borough,Area (sq mi),Population,Latitude,Longitude,Radius
0,Barking and Dagenham,13.93,212906,51.554117,0.150504,150.0
1,Barnet,33.49,395896,51.653090,-0.200226,232.0
2,Bexley,23.38,248287,51.441679,0.150488,194.0
3,Brent,16.70,329771,51.442026,0.231523,164.0
4,Bromley,57.97,332336,51.402805,0.014814,305.0
5,Camden,8.40,270029,51.542305,-0.139560,116.0
6,Croydon,33.41,386710,51.371305,-0.101957,232.0
7,Ealing,21.44,341806,51.512655,-0.305195,186.0
8,Enfield,31.74,333794,51.652085,-0.081018,226.0
9,Greenwich,18.28,287942,51.482084,-0.004542,172.0


We proceed to scrape coordinates leveraging on geocoding.

In [5]:
geolocator = Nominatim(user_agent="ny_explorer")
latitude_list=[]
longitude_list=[]

for borough in df['Borough']:
    address = '{}, UK'.format(borough)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    
df['Latitude']=latitude_list
df['Longitude']=longitude_list
df

,Borough,Area (sq mi),Population,Latitude,Longitude
0,Barking and Dagenham,13.93,212906,51.554117,0.150504
1,Barnet,33.49,395896,51.653090,-0.200226
2,Bexley,23.38,248287,51.441679,0.150488
3,Brent,16.70,329771,51.442026,0.231523
4,Bromley,57.97,332336,51.402805,0.014814
5,Camden,8.40,270029,51.542305,-0.139560
6,Croydon,33.41,386710,51.371305,-0.101957
7,Ealing,21.44,341806,51.512655,-0.305195
8,Enfield,31.74,333794,51.652085,-0.081018
9,Greenwich,18.28,287942,51.482084,-0.004542


We can now proceed to collect further information about the neighborhoods by setting Foursquare credentials (hidden cell) and by creating a function that retrieves information about venues.

In [6]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: UQPJ1CLSUMES5JPOU3P5GAPCLYPURBK2KFQ0SE24GU3AYXVT
CLIENT_SECRET:12ULPSLKA4LJXVJZZP1ZI323OF0LUKMT4BQC544QAQY3TSCE


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radius):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We now proceed to use the function to retrieve information and display the head of the resulting dataframe.

In [10]:
london_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                                    radius=df['Radius']
                                  )

Barking and Dagenham 
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich 
Hackney
Hammersmith and Fulham 
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [11]:
print(london_venues.shape)
london_venues.head()

(346, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barnet,51.65309,-0.200226,Ye Old Mitre Inne,51.652940,-0.199507,Pub
1,Barnet,51.65309,-0.200226,Joie de Vie,51.653659,-0.201288,Bakery
2,Barnet,51.65309,-0.200226,Caffè Nero,51.654861,-0.201743,Coffee Shop
3,Barnet,51.65309,-0.200226,Dudley's Pancake House,51.652965,-0.199625,Restaurant
4,Barnet,51.65309,-0.200226,Iceland,51.654776,-0.201687,Grocery Store


We filter the results for Coffee Shops, assuming that's the only relavant direct competition we face. Then, we group them by Borough and combine with the previous table.

In [12]:
coffee_venues=london_venues.loc[london_venues['Venue Category'].isin(['Coffee Shop'])]
coffee_venues

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Barnet,51.653090,-0.200226,Caffè Nero,51.654861,-0.201743,Coffee Shop
10,Barnet,51.653090,-0.200226,The Honey Bee,51.652501,-0.198851,Coffee Shop
28,Bromley,51.402805,0.014814,Caffè Nero,51.402653,0.015616,Coffee Shop
37,Bromley,51.402805,0.014814,Costa Coffee,51.404458,0.016645,Coffee Shop
48,Bromley,51.402805,0.014814,Starbucks,51.405300,0.015538,Coffee Shop
59,Croydon,51.371305,-0.101957,Crushed Bean,51.371000,-0.100176,Coffee Shop
76,Ealing,51.512655,-0.305195,Artisan Coffee,51.513024,-0.306655,Coffee Shop
81,Ealing,51.512655,-0.305195,Café Zee,51.513081,-0.305080,Coffee Shop
86,Ealing,51.512655,-0.305195,Harris + Hoole,51.513620,-0.303357,Coffee Shop
100,Ealing,51.512655,-0.305195,Starbucks,51.511098,-0.305196,Coffee Shop


In [13]:
coffee_grouped = coffee_venues.groupby('Borough').count().reset_index()
coffee_grouped=coffee_grouped.drop(columns=['Borough Latitude', 'Borough Longitude', 'Venue', 'Venue Latitude','Venue Longitude'])
coffee_grouped=coffee_grouped.rename(columns={"Venue Category": "Coffee Shops"})
coffee_grouped

,Borough,Coffee Shops
0,Barnet,2
1,Bromley,3
2,Croydon,1
3,Ealing,5
4,Enfield,5
5,Hammersmith and Fulham,2
6,Harrow,1
7,Havering,1
8,Hounslow,4
9,Kingston upon Thames,3


In [14]:
df_merged=pd.merge(left=df,right=coffee_grouped, how="left", on='Borough')
df_merged['Coffee Shops']=df_merged['Coffee Shops'].fillna(0)
df_merged

,Borough,Area (sq mi),Population,Latitude,Longitude,Radius,Coffee Shops
0,Barking and Dagenham,13.93,212906,51.554117,0.150504,150.0,0.0
1,Barnet,33.49,395896,51.653090,-0.200226,232.0,2.0
2,Bexley,23.38,248287,51.441679,0.150488,194.0,0.0
3,Brent,16.70,329771,51.442026,0.231523,164.0,0.0
4,Bromley,57.97,332336,51.402805,0.014814,305.0,3.0
5,Camden,8.40,270029,51.542305,-0.139560,116.0,0.0
6,Croydon,33.41,386710,51.371305,-0.101957,232.0,1.0
7,Ealing,21.44,341806,51.512655,-0.305195,186.0,5.0
8,Enfield,31.74,333794,51.652085,-0.081018,226.0,5.0
9,Greenwich,18.28,287942,51.482084,-0.004542,172.0,0.0


Finally, we compute our Expected Demand and identify the best potential Borough

In [15]:
df_merged['Expected Demand']=round(df_merged['Population']/(df_merged['Coffee Shops']+1),0)
df_merged

,Borough,Area (sq mi),Population,Latitude,Longitude,Radius,Coffee Shops,Expected Demand
0,Barking and Dagenham,13.93,212906,51.554117,0.150504,150.0,0.0,212906.0
1,Barnet,33.49,395896,51.653090,-0.200226,232.0,2.0,131965.0
2,Bexley,23.38,248287,51.441679,0.150488,194.0,0.0,248287.0
3,Brent,16.70,329771,51.442026,0.231523,164.0,0.0,329771.0
4,Bromley,57.97,332336,51.402805,0.014814,305.0,3.0,83084.0
5,Camden,8.40,270029,51.542305,-0.139560,116.0,0.0,270029.0
6,Croydon,33.41,386710,51.371305,-0.101957,232.0,1.0,193355.0
7,Ealing,21.44,341806,51.512655,-0.305195,186.0,5.0,56968.0
8,Enfield,31.74,333794,51.652085,-0.081018,226.0,5.0,55632.0
9,Greenwich,18.28,287942,51.482084,-0.004542,172.0,0.0,287942.0
